In [1]:
import pandas as pd
from pathlib import Path
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *
from tbdynamics.utils import build_contact_matrix
import plotly.graph_objects as go
from general_utils.parameter_utils import load_param_info
from general_utils.calibration_utils import round_sigfig
from general_utils.doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
    save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
new_calibration = False

In [2]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50]

In [5]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "infectious_seed": 1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
    'rr_progression_diabetes': 4.5
}

In [3]:
matrix = build_contact_matrix()

In [4]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    latent_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix,
    fixed_parameters
)

In [6]:
tb_model.run(params)

In [8]:
tb_model.get_outputs_df()

,susceptible,early_latent,late_latent,infectious,on_treatment,recovered
1800.0,1.499990e+05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
1801.0,5.742071e+04,1.051481e-03,5.462095e-04,1.439983e-01,5.550777e-02,6.601492e-02
1802.0,2.198098e+04,2.360954e-04,2.503573e-04,2.404329e-02,9.902392e-03,4.193154e-02
1803.0,8.414511e+03,4.201263e-05,6.611783e-05,4.569491e-03,1.641527e-03,1.841361e-02
1804.0,3.221543e+03,6.297656e-06,1.393316e-05,1.555965e-03,1.013523e-04,7.317716e-03
...,...,...,...,...,...,...
2016.0,7.764559e-86,-4.713048e-12,2.586041e-124,2.586041e-123,-9.783792e-10,9.471452e-95
2017.0,2.908407e-86,-4.713048e-12,7.042310e-125,7.083826e-124,-9.783792e-10,3.488651e-95
2018.0,1.089413e-86,-4.713048e-12,1.917763e-125,2.120349e-125,-9.783792e-10,1.284987e-95
2019.0,4.080651e-87,-4.713048e-12,5.222456e-126,4.142254e-126,-9.783792e-10,4.733034e-96


In [9]:
priors = [
    esp.UniformPrior("start_population_size", (150000, 300000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.02)),
    #UniformPrior("infectious_seed", [100, 2000]),
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    esp.UniformPrior("cdr_adjustment", (0.6, 1.0)),\
    esp. UniformPrior("rr_progression_diabetes", (2.0, 6.0))
    # UniformPrior("progression_multiplier", (0.1, 2.0)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    # UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    # UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    # UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    # UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    # UniformPrior("rr_progression_diabetes", [1, 10]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

targets = [
    est.TruncatedNormalTarget('notifications', notif, [0.0, np.inf], notif.max() * 0.1),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [11]:

with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')

TypeError: Found incompatible dtypes, <class 'numpy.int64'> and <class 'numpy.float64'>. Seen so far: [<class 'numpy.int64'>, <class 'numpy.float64'>, ..., ...]
Apply node that caused the error: BCMLogLike(start_population_size, contact_rate, rr_infection_latent, rr_infection_recovered, smear_positive_death_rate, smear_negative_death_rate, smear_positive_self_recovery, smear_negative_self_recovery, cdr_adjustment, rr_progression_diabetes)
Toposort index: 10
Inputs types: [TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=())]
Inputs shapes: [(), (), (), (), (), (), (), (), (), ()]
Inputs strides: [(), (), (), (), (), (), (), (), (), ()]
Inputs values: [array(227344.75719536), array(0.0094141), array(0.2015), array(0.2), array(0.3643377

In [ ]:
tb_model.run(parameters=params)
derived_df_0 = tb_model.get_derived_outputs_df()

In [ ]:
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [ ]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()
total_fig_name = "total.jpg"
fig2_3.write_image(SUPPLEMENT_PATH /total_fig_name)

In [ ]:
add_element_to_document("Outputs",FigElement(total_fig_name, caption="Notifications"), doc_sections)

In [ ]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()
notif_fig_name = "notifications.jpg"
notif_plot.write_image(SUPPLEMENT_PATH / notif_fig_name)

In [ ]:
add_element_to_document("Outputs",FigElement(notif_fig_name, caption="Notifications"), doc_sections)

In [ ]:
prev_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
prev_plot.show()
prev_plot.write_image(str(SUPPLEMENT_PATH) + "/prevalance.jpg")

In [ ]:
inci_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="incidence",
)
inci_plot.show()
inci_plot.write_image(str(SUPPLEMENT_PATH) + "/incidence.jpg")

In [ ]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Percentage latent"
)
latent_plot.show()
latent_plot.write_image(str(SUPPLEMENT_PATH) + "/latent.jpg")

In [ ]:
supplement = generate_doc("Supplemental Appendix", "austcovid")
compile_doc(doc_sections, supplement)

In [ ]:
# iterations = 20000
# burn_in = 2000
# n_chains = 20
# if new_calibration:
#     with pm.Model() as pm_model:
#         variables = epm.use_model(calibration_model)
#         idata_raw = pm.sample(step=[pm.DEMetropolis(variables)], draws=iterations, tune=0, cores=16, chains=n_chains)
#     idata_raw.to_netcdf(OUTPUT_PATH / "calibration_out.nc")
# else:
#     idata_raw = az.from_netcdf(OUTPUT_PATH / "calibration_out.nc")

# idata = idata_raw.sel(draw=range(burn_in, iterations))  # Discard burn-in

In [ ]:
idata_raw.posterior.isel(draw=0).to_dataframe()

In [ ]:
(idata.sample_stats.accepted.sum(axis=1) / idata.sample_stats.coords["draw"].size).to_dataframe()